In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
import os
from dotenv import load_dotenv


In [ ]:
load_dotenv()
google_api_key=os.getenv("GOOGLE_API_KEY")

class PloatWeaver:
    def __init__(self):
        """Initialize the VerseCraft agent with enhanced capabilities"""
        self.name= "PloatWeaverAgent"
        self.api_key = "google_api_key"
        
        
        self.llm = ChatGoogleGenerativeAI(
          model="gemini-2.0-flash",
          google_api_key=google_api_key,
          temperature=0.8,
         top_p=1.0,
         max_retries=4,
         max_tokens=500
              )
        
    def build_vectorstore(self, author_file_path, store_path):
        """Build a new vector store from a text file"""
        try:
            loader = TextLoader(author_file_path, encoding='utf-8')
            documents = loader.load()
            splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
            chunks = splitter.split_documents(documents)

            embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            vectorstore = FAISS.from_documents(chunks, embeddings)
            vectorstore.save_local(store_path)
            print(f"✅ Vector store saved at {store_path}")
            return True
        except Exception as e:
            print(f"❌ Error building vector store: {e}")
            return False    
       
        self.vector_stores = {}
    self.retrievers = {}

In [ ]:
# agents/plotweaver_agentic.py

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import os
from dotenv import load_dotenv

# ✅ Load Environment
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

# ✅ AGENT 1: Theme Analyzer
theme_prompt = PromptTemplate(
    input_variables=["theme"],
    template="""
You are a theme analysis agent. Your task is to break down the user's theme: "{theme}"
Output should include:
- Genre
- Emotional tone
- Setting suggestion
- Main internal conflict
Return in a structured format.
"""
)
theme_agent = LLMChain(llm=llm, prompt=theme_prompt)

# ✅ AGENT 2: Retriever Agent
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vstore = FAISS.load_local("./vectorstores/plot_ideas", embedding_model)
retriever = vstore.as_retriever()

def retrieve_context(theme_info: str):
    docs = retriever.get_relevant_documents(theme_info)
    return "\n\n".join([doc.page_content for doc in docs[:3]])

# ✅ AGENT 3: Plot Composer
plot_prompt = PromptTemplate(
    input_variables=["breakdown", "context", "theme"],
    template="""
You are PlotWeaver, an advanced plot generator.

Using:
- Theme breakdown: {breakdown}
- Retrieved literary context: {context}
- Original theme: {theme}

Create a 3-act plot with:
1. Title
2. Act I: Setup
3. Act II: Confrontation
4. Act III: Resolution
"""
)
plot_agent = LLMChain(llm=llm, prompt=plot_prompt)

# ✅ MAIN AGENTIC RAG FLOW
def agentic_generate_plot(theme):
    try:
        print("🧠 Agent 1: Analyzing theme...")
        theme_breakdown = theme_agent.run(theme)

        print("📚 Agent 2: Retrieving context...")
        context = retrieve_context(theme_breakdown)

        print("🧩 Agent 3: Composing plot...")
        final_plot = plot_agent.run({
            "theme": theme,
            "breakdown": theme_breakdown,
            "context": context
        })

        return final_plot

    except Exception as e:
        return f"❌ Agentic Plot Generation Failed: {str(e)}"

# ✅ TEST RUN
if __name__ == "__main__":
    theme = "reclaiming identity in a dystopian society"
    print("\n🎬 Agentic RAG Plot Output:\n")
    print(agentic_generate_plot(theme))


In [ ]:
def build_vectorstore(author_file_path, store_path):
    loader = TextLoader(author_file_path , encoding='utf-8')
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(store_path)
    print(f"Vector store saved at {store_path}")

making the vector stores 


In [32]:
build_vectorstore("./data/corpusForPloatweaver/brothersdostovesky.txt", "./vectorstores/plot_ideas")

Vector store saved at ./vectorstores/plot_ideas


In [33]:
build_vectorstore("./data/corpusForPloatweaver/crimesandpunishments.txt", "./vectorstores/plot_ideas")

Vector store saved at ./vectorstores/plot_ideas


In [40]:
build_vectorstore("./data/corpusForPloatweaver/warandpeace.txt", "./vectorstores/plot_ideas")

Vector store saved at ./vectorstores/plot_ideas


In [39]:
build_vectorstore("./data/corpusForPloatweaver/taleoftwocities.txt", "./vectorstores/plot_ideas")

Vector store saved at ./vectorstores/plot_ideas


In [ ]:
build_vectorstore("data/corpusForPloatweaver/the idiot.txt", "./vectorstores/vectorstoreforploats/idiot")

In [ ]:
build_vectorstore("./data/corpusForPloatweaver/pride and prejudice.txt", "./vectorstores/vectorstoreforploats/prideandprejudice")

In [ ]:
build_vectorstore("./data/corpusForPloatweaver/kafka_trial.txt", "./vectorstores/vectorstoreforploats/trail")

In [ ]:
#some oputput from our cluaiddieee 

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import os
from dotenv import load_dotenv
from typing import Dict, Optional
from pathlib import Path
class PlotWeaver:
    """
    An advanced agentic RAG system for generating creative plots based on user preferences.
    
    Uses three specialized agents:
    1. Theme Analyzer - Breaks down user inputs into structured theme components
    2. Context Retriever - Finds relevant literary examples and patterns
    3. Plot Composer - Generates final structured plot
    """
    
    def __init__(self, vectorstore_path: str = "./vectorstores/plot_ideas"):
        """
        Initialize the PlotWeaver system.
        
        Args:
            vectorstore_path: Path to the FAISS vectorstore containing plot ideas
        """
        self.vectorstore_path = vectorstore_path
        self._setup_environment()
        self._initialize_llm()
        self._setup_agents()
        self._load_vectorstore()
    
    def _setup_environment(self):
        """Load environment variables and API keys."""
        load_dotenv()
        self.google_api_key = os.getenv("GOOGLE_API_KEY")
        
        if not self.google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in environment variables")
    
    def _initialize_llm(self):
        """Initialize the Google Generative AI model."""
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash", 
            google_api_key=self.google_api_key
        )
    
    def _setup_agents(self):
        """Setup the three specialized agents with their prompts."""
        
        # Agent 1: Theme Analyzer
        self.theme_prompt = PromptTemplate(
            input_variables=["genre", "mood", "complexity"],
            template="""
You are a sophisticated theme analysis agent. Analyze the user's creative preferences:

Genre: {genre}
Desired Mood: {mood}
Plot Complexity: {complexity}

Based on these inputs, provide a structured analysis including:
- Primary genre characteristics and conventions
- Emotional tone and atmosphere
- Suggested setting and world-building elements
- Main character archetype suggestions
- Core internal/external conflict types
- Thematic depth based on complexity level

Format your response as a clear, structured analysis that will guide plot generation.

"""
        )
        # Modern LangChain: prompt | llm | output_parser
        self.theme_agent = self.theme_prompt | self.llm | StrOutputParser()
        
        # Agent 3: Plot Composer
        self.plot_prompt = PromptTemplate(
            input_variables=["analysis", "context", "genre", "mood", "complexity"],
            template="""
You are PlotWeaver, an advanced plot generation specialist. You can create stories that are imaginative yet a blend of facts and logic , consider yourself a great player of words .
User Requirements:

- Genre: {genre}
- Mood: {mood}
- Complexity: {complexity}

Theme Analysis: {analysis}

Retrieved Literary Context: {context}

Generate a compelling plot with the following structure:

**TITLE:** [Creative, genre-appropriate title]

**PLOT SUMMARY:**
[2-3 sentence overview]

**ACT I - SETUP:**
[Detailed setup including character introduction, world establishment, and inciting incident]

**ACT II - CONFRONTATION:**
[Rising action, conflicts, obstacles, and character development]

**ACT III - RESOLUTION:**
[Climax, resolution, and character transformation]

**KEY THEMES:**
[Major themes explored in the story]

**CHARACTER ARC:**
[Brief description of protagonist's journey]

Ensure the complexity matches the user's request: simple plots should be straightforward, while complex plots should include subplots, multiple character arcs, and layered themes.
"""
        )
        # Modern LangChain: prompt | llm | output_parser
        self.plot_agent = self.plot_prompt | self.llm | StrOutputParser()
    
    def _load_vectorstore(self):
        """Load the FAISS vectorstore for context retrieval."""
        try:
            self.embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            self.vectorstore = FAISS.load_local(
                self.vectorstore_path, 
                self.embedding_model,
                allow_dangerous_deserialization=True
            )
            self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 5})
            print(f"✅ Vectorstore loaded successfully from {self.vectorstore_path}")
        except Exception as e:
            print(f"⚠️ Warning: Could not load vectorstore from {self.vectorstore_path}")
            print(f"Error: {str(e)}")
            print("🔧 You need to create the vectorstore first. See setup instructions below.")
            self.vectorstore = None
            self.retriever = None
    
    def _retrieve_context(self, theme_analysis: str) -> str:
        """
        Retrieve relevant context from the vectorstore.
        
        Args:
            theme_analysis: The analyzed theme information
            
        Returns:
            Concatenated relevant document content
        """
        if not self.retriever:
            return "No vectorstore context available."
        
        try:
            docs = self.retriever.get_relevant_documents(theme_analysis)
            context = "\n\n".join([doc.page_content for doc in docs[:3]])
            return context if context else "No relevant context found."
        except Exception as e:
            return f"Error retrieving context: {str(e)}"
    
    def generate_plot(self, genre: str, desired_mood: str, plot_complexity: str) -> str:
        """
        Generate a complete plot using the agentic RAG system.
        
        Args:
            genre: The story genre (e.g., "science fiction", "fantasy", "mystery")
            desired_mood: The emotional tone (e.g., "dark", "hopeful", "suspenseful")
            plot_complexity: Complexity level ("simple", "moderate", "complex")
            
        Returns:
            Generated plot as a formatted string
        """
        try:
            print("🧠 Agent 1: Analyzing theme and preferences...")
            theme_analysis = self.theme_agent.invoke({
                "genre": genre,
                "mood": desired_mood,
                "complexity": plot_complexity
            })
            
            print("📚 Agent 2: Retrieving relevant context...")
            context = self._retrieve_context(theme_analysis)
            
            print("🧩 Agent 3: Composing final plot...")
            final_plot = self.plot_agent.invoke({
                "genre": genre,
                "mood": desired_mood,
                "complexity": plot_complexity,
                "analysis": theme_analysis,
                "context": context
            })
            
            return final_plot
            
        except Exception as e:
            return f"❌ Plot Generation Failed: {str(e)}"
    
    def create_vectorstore(self, documents_path: str):
        """
        Create a vectorstore from text documents.
        
        Args:
            documents_path: Path to directory containing text files with plot ideas
        """
        try:
            # Load documents
            documents = []
            for filename in os.listdir(documents_path):
                if filename.endswith('.txt'):
                    file_path = str(Path(documents_path) / filename).replace('\\', '/')
                    loader = TextLoader(file_path)
                    docs = loader.load()
                    documents.extend(docs)
            
            # Split documents
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            splits = text_splitter.split_documents(documents)
            
            # Create vectorstore
            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            vectorstore = FAISS.from_documents(splits, embedding_model)
            
            # Save vectorstore
            os.makedirs(os.path.dirname(self.vectorstore_path), exist_ok=True)
            vectorstore.save_local(self.vectorstore_path)
            
            print(f"✅ Vectorstore created and saved to {self.vectorstore_path}")
            
            # Reload the vectorstore
            self._load_vectorstore()
            
        except Exception as e:
            print(f"❌ Error creating vectorstore: {str(e)}")


# ✅ USAGE EXAMPLE
if __name__ == "__main__":
    # Initialize PlotWeaver
    plot_weaver = PlotWeaver()
    
    
    # Example usage
    genre = "science fiction"
    mood = "dark and mysterious"
    complexity = "complex"
    
    print(f"\n🎬 Generating plot for:")
    print(f"📖 Genre: {genre}")
    print(f"🎭 Mood: {mood}")
    print(f"🧩 Complexity: {complexity}")
    print("\n" + "="*50 + "\n")
    
    # Generate plot
    result = plot_weaver.generate_plot(genre, mood, complexity)
    print(result)


# ✅ VECTORSTORE SETUP INSTRUCTIONS
"""
VECTORSTORE SETUP INSTRUCTIONS:

1. Create a directory structure:
   project/
   ├── vectorstores/
   │   └── plot_ideas/
   ├── documents/
   │   ├── fantasy_plots.txt
   │   ├── scifi_plots.txt
   │   ├── mystery_plots.txt
   │   └── ... (more plot idea files)
   └── your_main_script.py

2. Populate the documents/ folder with text files containing:
   - Plot summaries
   - Character archetypes
   - Story structures
   - Genre conventions
   - Narrative techniques

3. Create the vectorstore by running:
   ```python
   plot_weaver = PlotWeaver()
   plot_weaver.create_vectorstore("./documents")
   ```

4. Example content for documents/fantasy_plots.txt:
   ```
   The Hero's Journey: A young protagonist discovers hidden powers and must save the world.
   The Chosen One: An unlikely hero is prophesied to defeat an ancient evil.
   Portal Fantasy: Character travels to another world through magical means.
   Dark Fantasy: Heroes face morally ambiguous choices in a gritty fantasy setting.
   ```

The vectorstore will be automatically created at ./vectorstores/plot_ideas/
"""

✅ Vectorstore loaded successfully from ./vectorstores/plot_ideas

🎬 Generating plot for:
📖 Genre: science fiction
🎭 Mood: dark and mysterious
🧩 Complexity: complex


🧠 Agent 1: Analyzing theme and preferences...
📚 Agent 2: Retrieving relevant context...


C:\Users\shiva\AppData\Local\Temp\ipykernel_8112\2727369433.py:154: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(theme_analysis)


🧩 Agent 3: Composing final plot...
Okay, PlotWeaver is online and ready to weave a dark and mysterious science fiction plot with a complex structure. Here's the generated plot outline:

**TITLE:** *Echo Chamber of Stars*

**PLOT SUMMARY:** In a far-future society where memories can be extracted, traded, and relived, a "Memory Broker" stumbles upon a fragmented recollection that hints at a universe-shattering conspiracy. Her pursuit of the truth leads her down a rabbit hole of manipulated realities and forgotten histories, forcing her to question the very nature of her existence and the society she serves.

**ACT I - SETUP:**

*   **Introduction of Elara Vance:** Elara is a highly skilled Memory Broker, operating in the sprawling, neon-drenched megacity of Neo-Alexandria on the terraformed planet of Kepler-186f. She's cynical and detached, viewing memories as commodities rather than personal experiences. Her workspace is a sleek, sterile environment, reflecting her professional detachme

'\nVECTORSTORE SETUP INSTRUCTIONS:\n\n1. Create a directory structure:\n   project/\n   ├── vectorstores/\n   │   └── plot_ideas/\n   ├── documents/\n   │   ├── fantasy_plots.txt\n   │   ├── scifi_plots.txt\n   │   ├── mystery_plots.txt\n   │   └── ... (more plot idea files)\n   └── your_main_script.py\n\n2. Populate the documents/ folder with text files containing:\n   - Plot summaries\n   - Character archetypes\n   - Story structures\n   - Genre conventions\n   - Narrative techniques\n\n3. Create the vectorstore by running:\n   ```python\n   plot_weaver = PlotWeaver()\n   plot_weaver.create_vectorstore("./documents")\n   ```\n\n4. Example content for documents/fantasy_plots.txt:\n   ```\n   The Hero\'s Journey: A young protagonist discovers hidden powers and must save the world.\n   The Chosen One: An unlikely hero is prophesied to defeat an ancient evil.\n   Portal Fantasy: Character travels to another world through magical means.\n   Dark Fantasy: Heroes face morally ambiguous choi

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import os
from dotenv import load_dotenv
from typing import Dict, Optional, List
from pathlib import Path
import json

class PlotWeaver:
    """
    An advanced agentic RAG system for generating creative plots based on user preferences.
    
    Uses three specialized agents:
    1. Theme Analyzer - Breaks down user inputs into structured theme components
    2. Context Retriever - Finds relevant literary examples and patterns
    3. Plot Composer - Generates final structured plot
    """
    
    def __init__(self, vectorstore_path: str = "./vectorstores/plot_ideas"):
        """
        Initialize the PlotWeaver system.
        
        Args:
            vectorstore_path: Path to the FAISS vectorstore containing plot ideas
        """
        self.vectorstore_path = vectorstore_path
        self._setup_environment()
        self._initialize_llm()
        self._setup_agents()
        self._load_vectorstore()
        self._initialize_options()
    
    def _setup_environment(self):
        """Load environment variables and API keys."""
        load_dotenv()
        self.google_api_key = os.getenv("GOOGLE_API_KEY")
        
        if not self.google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in environment variables")
    
    def _initialize_llm(self):
        """Initialize the Google Generative AI model."""
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash", 
            google_api_key=self.google_api_key
        )
    
    def _initialize_options(self):
        """Initialize available options for genres, moods, and complexity levels."""
        self.available_genres = [
            "Romance",  "Fantasy", "Mystery", "Thriller",
            "Horror", "Comedy", "Drama", "Action", "Adventure", "Western",
            "Slice_of_life" 
            
        ]
        
        self.available_moods = [
            "Dark", "Light", "Hopeful", "Melancholic", "Suspenseful",
            "Romantic", "Humorous", "Mysterious", "Uplifting", "Intense",
            "Peaceful", "Nostalgic", "Adventurous", "Dreamy", "Gritty",
            "Joyful" ,"Passionate","Serene" , "Whimsical"
        ] 
        
        self.complexity_levels = {
            "simple": "Single protagonist, linear plot, one main conflict",
            "moderate": "Multiple characters, subplots, character development arcs",
            "complex": "Multiple POVs, intricate plotting, layered themes, complex world-building"
        }
    
    def _setup_agents(self):
        """Setup the three specialized agents with their prompts."""
        
        # Agent 1: Theme Analyzer
        self.theme_prompt = PromptTemplate(
            input_variables=["genre", "mood", "complexity"],
            template="""
You are a sophisticated theme analysis agent. Analyze the user's creative preferences:

Genre: {genre}
Desired Mood: {mood}
Plot Complexity: {complexity}

Based on these inputs, provide a structured analysis including:
- Primary genre characteristics and conventions
- Emotional tone and atmosphere
- Suggested setting and world-building elements
- Main character archetype suggestions
- Core internal/external conflict types
- Thematic depth based on complexity level

Format your response as a clear, structured analysis that will guide plot generation.
"""
        )
        self.theme_agent = self.theme_prompt | self.llm | StrOutputParser()
        
        # Agent 3: Plot Composer
        self.plot_prompt = PromptTemplate(
            input_variables=["analysis", "context", "genre", "mood", "complexity"],
            template="""
You are PlotWeaver, an advanced plot generation specialist. You can create stories that are imaginative yet a blend of facts and logic, consider yourself a great player of words.

User Requirements:
- Genre: {genre}
- Mood: {mood}
- Complexity: {complexity}

Theme Analysis: {analysis}

Retrieved Literary Context: {context}

Generate a compelling plot with the following structure:

**TITLE:** [Creative, genre-appropriate title]

**PLOT SUMMARY:**
[2-3 sentence overview]

**ACT I - SETUP:**
[Detailed setup including character introduction, world establishment, and inciting incident]

**ACT II - CONFRONTATION:**
[Rising action, conflicts, obstacles, and character development]

**ACT III - RESOLUTION:**
[Climax, resolution, and character transformation]

**KEY THEMES:**
[Major themes explored in the story]

**CHARACTER ARC:**
[Brief description of protagonist's journey]

Ensure the complexity matches the user's request: simple plots should be straightforward, while complex plots should include subplots, multiple character arcs, and layered themes.
"""
        )
        self.plot_agent = self.plot_prompt | self.llm | StrOutputParser()
    
    def _load_vectorstore(self):
        """Load the FAISS vectorstore for context retrieval."""
        try:
            self.embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            self.vectorstore = FAISS.load_local(
                self.vectorstore_path, 
                self.embedding_model,
                allow_dangerous_deserialization=True
            )
            self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 5})
            print(f"✅ Vectorstore loaded successfully from {self.vectorstore_path}")
        except Exception as e:
            print(f"⚠️ Warning: Could not load vectorstore from {self.vectorstore_path}")
            print(f"Error: {str(e)}")
            print("🔧 You need to create the vectorstore first. See setup instructions below.")
            self.vectorstore = None
            self.retriever = None
    
    def _retrieve_context(self, theme_analysis: str) -> str:
        """
        Retrieve relevant context from the vectorstore.
        
        Args:
            theme_analysis: The analyzed theme information
            
        Returns:
            Concatenated relevant document content
        """
        if not self.retriever:
            return "No vectorstore context available."
        
        try:
            docs = self.retriever.get_relevant_documents(theme_analysis)
            context = "\n\n".join([doc.page_content for doc in docs[:3]])
            return context if context else "No relevant context found."
        except Exception as e:
            return f"Error retrieving context: {str(e)}"
    
    def generate_plot(self, genre: str, desired_mood: str, plot_complexity: str) -> str:
        """
        Generate a complete plot using the agentic RAG system.
        
        Args:
            genre: The story genre (e.g., "science fiction", "fantasy", "mystery")
            desired_mood: The emotional tone (e.g., "dark", "hopeful", "suspenseful")
            plot_complexity: Complexity level ("simple", "moderate", "complex")
            
        Returns:
            Generated plot as a formatted string
        """
        try:
            print("🧠 Agent 1: Analyzing theme and preferences...")
            theme_analysis = self.theme_agent.invoke({
                "genre": genre,
                "mood": desired_mood,
                "complexity": plot_complexity
            })
            
            print("📚 Agent 2: Retrieving relevant context...")
            context = self._retrieve_context(theme_analysis)
            
            print("🧩 Agent 3: Composing final plot...")
            final_plot = self.plot_agent.invoke({
                "genre": genre,
                "mood": desired_mood,
                "complexity": plot_complexity,
                "analysis": theme_analysis,
                "context": context
            })
            
            return final_plot
            
        except Exception as e:
            return f"❌ Plot Generation Failed: {str(e)}"
    
    def create_vectorstore(self, documents_path: str):
        """
        Create a vectorstore from text documents.
        
        Args:
            documents_path: Path to directory containing text files with plot ideas
        """
        try:
            # Load documents
            documents = []
            for filename in os.listdir(documents_path):
                if filename.endswith('.txt'):
                    file_path = str(Path(documents_path) / filename).replace('\\', '/')
                    loader = TextLoader(file_path)
                    docs = loader.load()
                    documents.extend(docs)
            
            # Split documents
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            splits = text_splitter.split_documents(documents)
            
            # Create vectorstore
            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            vectorstore = FAISS.from_documents(splits, embedding_model)
            
            # Save vectorstore
            os.makedirs(os.path.dirname(self.vectorstore_path), exist_ok=True)
            vectorstore.save_local(self.vectorstore_path)
            
            print(f"✅ Vectorstore created and saved to {self.vectorstore_path}")
            
            # Reload the vectorstore
            self._load_vectorstore()
            
        except Exception as e:
            print(f"❌ Error creating vectorstore: {str(e)}")

    def get_available_genres(self) -> List[str]:
        """Return list of available genres."""
        return self.available_genres

    def get_available_moods(self) -> List[str]:
        """Return list of available moods."""
        return self.available_moods

    def get_complexity_info(self) -> Dict[str, str]:
        """Return complexity levels with descriptions."""
        return self.complexity_levels

    def process_request(self, request: Dict) -> Dict:
        """Process a request from the agent manager or UI"""
        try:
            action = request.get('action', 'generate_plot')
            
            if action == 'generate_plot':
                genre = request.get('genre', 'Romance')
                mood = request.get('mood', 'Neutral')
                complexity = request.get('complexity', 'simple')
                
                plot = self.generate_plot(genre, mood, complexity)
                return {
                    'status': 'success',
                    'plot': plot,
                    'parameters': {
                        'genre': genre,
                        'mood': mood,
                        'complexity': complexity
                    }
                }
            
            elif action == 'get_genres':
                return {
                    'status': 'success',
                    'genres': self.get_available_genres()
                }
            
            elif action == 'get_moods':
                return {
                    'status': 'success',
                    'moods': self.get_available_moods()
                }
            
            elif action == 'get_complexity':
                return {
                    'status': 'success',
                    'complexity_info': self.get_complexity_info()
                }
            
            elif action == 'health_check':
                return {
                    'status': 'success',
                    'message': 'PlotWeaver is running',
                    'vectorstore_loaded': self.vectorstore is not None
                }
            
            else:
                return {
                    'status': 'error',
                    'error': f'Unknown action: {action}'
                }
                
        except Exception as e:
            return {
                'status': 'error',
                'error': f'Request processing failed: {str(e)}'
            }

    def validate_inputs(self, genre: str, mood: str, complexity: str) -> Dict:
        """Validate user inputs and return validation result."""
        errors = []
        
        if genre not in self.available_genres:
            errors.append(f"Invalid genre. Available genres: {', '.join(self.available_genres)}")
        
        if mood not in self.available_moods:
            errors.append(f"Invalid mood. Available moods: {', '.join(self.available_moods)}")
        
        if complexity not in self.complexity_levels:
            errors.append(f"Invalid complexity. Available levels: {', '.join(self.complexity_levels.keys())}")
        
        return {
            'valid': len(errors) == 0,
            'errors': errors
        }

# Example usage and testing functions
def main():
    """Example usage of PlotWeaver system."""
    try:
        # Initialize PlotWeaver
        plot_weaver = PlotWeaver()
        
        # Example request
        request = {
            'action': 'generate_plot',
            'genre': 'Romance',
            'mood': 'Passionate',
            'complexity': 'complex'
        }
        
        # Process request
        result = plot_weaver.process_request(request)
        
        if result['status'] == 'success':
            print("Generated Plot:")
            print("=" * 50)
            print(result['plot'])
        else:
            print(f"Error: {result['error']}")
            
    except Exception as e:
        print(f"Failed to initialize PlotWeaver: {str(e)}")

if __name__ == "__main__":
    main()

✅ Vectorstore loaded successfully from ./vectorstores/plot_ideas
🧠 Agent 1: Analyzing theme and preferences...
📚 Agent 2: Retrieving relevant context...
🧩 Agent 3: Composing final plot...
Generated Plot:
Okay, here's a plot outline for a passionate romance with a complex plot, drawing inspiration from the provided literary context (the themes of societal expectations, duty vs. desire, and the pain of separation are particularly relevant) and your earlier thematic analysis.

**TITLE:** *The Gilded Cage of Petals*

**PLOT SUMMARY:** In Belle Époque Paris, a talented but impoverished artist, Éloïse, finds herself entangled in a passionate affair with a powerful, married Duke, Armand. As their love deepens, they must navigate the treacherous waters of societal expectations, political intrigue, and Armand's own dark secrets, ultimately forcing Éloïse to choose between her heart's desire and her artistic integrity.

**ACT I - SETUP:**

*   **Introduction:** We meet Éloïse Moreau, a gifted bu